In [1]:
# %%python -m pip install --upgrade pip setuptools
%pip install tensorflow
# %pip install tensorflow-gpu
%pip install scipy
%pip install pillow
%pip install opencv-python

In [2]:
import os
# import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import cv2

from google.colab import files
src = list(files.upload().values())[0]
open('helper_functions.py','wb').write(src)
import helper_functions
src1 = list(files.upload().values())[0]
open('helper.py','wb').write(src1)
import helper
import tqdm
import matplotlib.pyplot as plt

Instructions for updating:
non-resource variables are not supported in the long term


Saving helper_functions.py to helper_functions.py


Saving helper.py to helper.py


In [3]:
NUM_CLASSES = 2
EPOCHS = 17
BATCH_SIZE = 2
DROPOUT = 0.75
image_shape = (3584, 3584)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [4]:
import re
import random
import numpy as np
import os.path
import scipy.misc
import shutil
import zipfile
import time
# import tensorflow as tf
from glob import glob
from urllib.request import urlretrieve
from tqdm import tqdm
import cv2
from PIL import Image
class DLProgress(tqdm):
    last_block = 0

    def hook(self, block_num=1, block_size=1, total_size=None):
        self.total = total_size
        self.update((block_num - self.last_block) * block_size)
        self.last_block = block_num

In [5]:
def maybe_download_pretrained_vgg(data_dir):
    """
    Download and extract pretrained vgg model if it doesn't exist
    :param data_dir: Directory to download the model to
    """
    vgg_filename = 'vgg.zip'
    vgg_path = os.path.join(data_dir, 'vgg')
    vgg_files = [
        os.path.join(vgg_path, 'variables/variables.data-00000-of-00001'),
        os.path.join(vgg_path, 'variables/variables.index'),
        os.path.join(vgg_path, 'saved_model.pb')]

    missing_vgg_files = [vgg_file for vgg_file in vgg_files if not os.path.exists(vgg_file)]
    if missing_vgg_files:
        # Clean vgg dir
        if os.path.exists(vgg_path):
            shutil.rmtree(vgg_path)
        os.makedirs(vgg_path)

        # Download vgg
        print('Downloading pre-trained vgg model...')
        with DLProgress(unit='B', unit_scale=True, miniters=1) as pbar:
            urlretrieve(
                'https://s3-us-west-1.amazonaws.com/udacity-selfdrivingcar/vgg.zip',
                os.path.join(vgg_path, vgg_filename),
                pbar.hook)

        # Extract vgg
        print('Extracting model...')
        zip_ref = zipfile.ZipFile(os.path.join(vgg_path, vgg_filename), 'r')
        zip_ref.extractall(data_dir)
        zip_ref.close()

        # Remove zip file to save space
        os.remove(os.path.join(vgg_path, vgg_filename))

In [7]:
# Placeholders
correct_label = tf.placeholder(tf.float32, [None, image_shape[0], image_shape[1], NUM_CLASSES])
learning_rate = tf.placeholder(tf.float32)
keep_prob = tf.placeholder(tf.float32)

# Directories
root = 'data'
rgb_dir = root + '/train/image'
gt_dir = root + '/train/label'


vgg_path = 'vgg' # When downloading VGG model
runs_dir = './runs'

# Comment this when downloaded VGG model
# maybe_download_pretrained_vgg(vgg_path)
'''
Get image by concatenating <dir_name> and <file_from_list>
Example: image = cv2.imread(rgb_dir + rgb_list[0], 1) to read first image from RGB
'''

997MB [00:45, 22.1MB/s]                           


Extracting model...


'\nGet image by concatenating <dir_name> and <file_from_list>\nExample: image = cv2.imread(rgb_dir + rgb_list[0], 1) to read first image from RGB\n'

In [8]:
vgg_path = '/content/vgg/vgg' # Uncomment this when you have downloaded

In [9]:
def gen_batch_function(data_folder, image_shape):
    """
    Generate function to create batches of training data
    :param data_folder: Path to folder that contains all the datasets
    :param image_shape: Tuple - Shape of image
    :return:
    """
    def get_batches_fn(batch_size):
        """
        Create batches of training data
        :param batch_size: Batch Size
        :return: Batches of training data
        """
        image_paths = glob(os.path.join(data_folder, 'train', 'image', '*.png'))
        if(len(image_paths) > 0):
            print('Found {0} training images'.format(len(image_paths)))
        else:
            print('Something is wrong. I couldn\'t see any images')
        label_paths = {
            re.sub(r'_vis', '', os.path.basename(path)): path
            for path in glob(os.path.join(data_folder, 'train', 'label', '*.png'))}

        if(len(label_paths) > 0):
            print('Found {0} training labels'.format(len(label_paths)))
        else:
            print('Something is wrong. I couldn\'t see any images')
        background_color = np.array([0, 0, 0])

        random.shuffle(image_paths)
        for batch_i in range(0, len(image_paths), batch_size):
            images = []
            gt_images = []
            for image_file in image_paths[batch_i:batch_i+batch_size]:
                gt_image_file = label_paths[os.path.basename(image_file)]

                image = scipy.misc.imresize(scipy.misc.imread(image_file), image_shape)
                gt_image = scipy.misc.imresize(scipy.misc.imread(gt_image_file), image_shape)
                print(image.shape, gt_image.shape)
                gt_bg = np.all(gt_image == background_color, axis=2)
                gt_bg = gt_bg.reshape(*gt_bg.shape, 1)
                gt_image = np.concatenate((gt_bg, np.invert(gt_bg)), axis=2)

                images.append(image)
                gt_images.append(gt_image)

            yield np.array(images), np.array(gt_images)
    return get_batches_fn

In [18]:
import tensorflow

In [19]:
def load_vgg(sess, vgg_path):
    #helper.maybe_download_pretrained_vgg(vgg_path)
    model = tf.saved_model.loader.load(sess, ['vgg16'], vgg_path)
    print('VGG16 Loaded')
    graph = tf.get_default_graph()

    image_input = graph.get_tensor_by_name('image_input:0')
    layer_3 = graph.get_tensor_by_name('layer3_out:0')
    layer_4 = graph.get_tensor_by_name('layer4_out:0')
    layer_7 = graph.get_tensor_by_name('layer7_out:0')
    keep_prob = graph.get_tensor_by_name('keep_prob:0')

    return image_input, keep_prob, layer_3, layer_4, layer_7


def layers(vgglayer3_out, vgglayer4_out, vgglayer7_out, num_classes):


    ''' Better naming '''
    layer3, layer4, layer7 = vgglayer3_out, vgglayer4_out, vgglayer7_out

    '''
    You would conventionally apply a FC layer on top of layer 7.
    Do a 1x1 pooling and deduce fcn8
    TODO: a little more on 1x1 conv. What it does?
    '''

    fcn8 = tensorflow.keras.layers(layer7, filters = num_classes, kernel_size = 1, name = 'fcn8')

    '''
    Upsample fcn8 to get fcn9.
    Match size of fcn9 to layer4 to aid adding a skip connection
    This can be done by inferring size from layer4
    '''
    fcn9 = tensorflow.keras.layers.conv2d_transpose(fcn8, filters = layer4.get_shape().as_list()[-1],
                        padding = 'SAME', kernel_size = 4, strides = (2, 2), name = 'fcn9')

    '''Add Skip connection from layer4 to fcn9'''
    fcn9_skip_connected = tf.add(layer4, fcn9, name = 'fcn9__skip_connected')

    '''Upsample to fcn10, keeping in mind dimensions of layer3'''
    fcn10 = tensorflow.keras.layers.conv2d_transpose(fcn9_skip_connected, filters = layer3.get_shape().as_list()[-1],
                        padding = 'SAME', kernel_size = 4, strides = (2, 2), name = 'fcn10')

    '''Add skip connection layer3+fcn10'''
    fcn10_skip_connected = tf.add(fcn10, layer3, name = 'fcn10_skip_connected')

    '''
    Upsample to match final image size
    TODO: Why is filters = num_classes?
    Found: the output map will contain two plots: for ROAD, and NOT ROAD
    '''
    fcn11 = tensorflow.keras.layersconv2d_transpose(fcn10_skip_connected, filters=num_classes,
                                        kernel_size=16, strides=(8, 8), padding='SAME', name="fcn11")
    return fcn11

def optimize(nn_last_layer, correct_label, learning_rate, num_classes):

    '''Reshape tensors to 2d, pixel<->class mapping'''
    logits = tf.reshape(nn_last_layer, (-1, num_classes), name = 'fcn_logits')

    gt_reshaped = tf.reshape(correct_label, (-1, num_classes), name = 'gt_reshaped')

    '''Calculate loss'''
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits = logits, labels = gt_reshaped)
    loss_op = tf.reduce_mean(cross_entropy, name = 'fcn_loss')

    train_op = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss_op)

    return logits, loss_op, train_op

def train_nn(sess, epochs, batch_size, get_batches_fn,
            train_op, loss_op, input_image, correct_label,
            keep_prob, learning_rate):

    keep_prob_value = 0.5
    learning_rate_value = 0.0001

    for epoch in range(epochs):
        '''Atleast initially'''
        total_loss = 0
        for X_batch, gt_batch in get_batches_fn(batch_size):
            loss, _ = sess.run([loss_op, train_op],
            feed_dict = {input_image: X_batch, correct_label: gt_batch,
            learning_rate: learning_rate_value, keep_prob: keep_prob_value})

            total_loss += loss

        print('Epoch {}...'.format(epoch+1))
        print('Loss {:.3f}'.format(total_loss))
        print()

In [11]:
def save_inference_samples(runs_dir, data_dir, sess, image_shape, logits, keep_prob, input_image):
    # Make folder for current run
    output_dir = os.path.join(runs_dir, str(time.time()))
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)
    os.makedirs(output_dir)

    # Run NN on test images and save them to HD
    print('Training Finished. Saving test images to: {}'.format(output_dir))
    image_outputs = gen_test_output(
        sess, logits, keep_prob, input_image, os.path.join(data_dir, 'test'), image_shape)
    for name, image in image_outputs:
        cv2.imwrite(os.path.join(output_dir, name), image)

In [20]:
from tensorflow.keras.layers import Conv2D

In [ ]:
get_batches_fn = gen_batch_function(root, image_shape)
with tf.Session() as sess:

        '''
        Get the required layers from pretrained model
        '''
        input_tensor, keep_prob, layer3, layer4, layer7 = load_vgg(sess, vgg_path)

        '''
        Obtain upsampled output layer from these three, skip connected layers
        returns: fcn11
        '''
        model_output = layers(layer3, layer4, layer7, NUM_CLASSES)

        '''
        Compile model and evaluate loss
        Then Train
        '''

        logits, loss_op, train_op = optimize(model_output, correct_label, learning_rate, NUM_CLASSES)
        sess.run(tf.global_variables_initializer())
        sess.run(tf.local_variables_initializer())
        print("Model Built")
        helper_functions.train_nn(sess, EPOCHS, BATCH_SIZE, get_batches_fn,
                train_op, loss_op, input_tensor, correct_label, keep_prob, learning_rate)
        print("Finished training")

        helper.save_inference_samples(runs_dir, 'data/test', sess, image_shape, logits, keep_prob, input_tensor)